<a href="https://colab.research.google.com/github/kangwonlee/2018pycpp/blob/ci-update/30.pointers-and-memory-management/00.pointers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Introducing Pointers



* Using pointers, programs in C/C++ languages can read and/or write to any memory with right permission.



* Pointer variables contain integer values of specific addresses.



* Let's consider following table.



In [ ]:
import random

import numpy.random as nr
import pandas as pd



In [ ]:
base_addr = 0x60000000

# Size of one variable : 4 Bytes == 32 bits
size_t = 4

# Number of 4 Byte variables
no_cell = 8


memory_map = pd.DataFrame(
    data={
        "index": range(no_cell)
    }
)

memory_map["offset"] = memory_map["index"] * size_t
memory_map["offset (hex)"] = memory_map["offset"].apply(lambda i:f"{i:02x}")
memory_map["address"] = memory_map["offset"].add(base_addr)
memory_map["address (hex)"] = memory_map["address"].apply(lambda i:f"{i:08x}")
memory_map["content"] = nr.randint(0xa000000, 0x7fffffff, size=(no_cell, 1))
memory_map["content (hex)"] = memory_map["content"].apply(lambda i:f"{i:08x}")

memory_map.set_index("address (hex)", inplace=True)
memory_map[["content (hex)"]]



* This tries to represent a part of the memory.
* It is visible to a program.
* The contents have random values because no initialization yet.



* Let's assume we store a value `0` at `0x6000 0010`.



In [ ]:
memory_map.loc["60000010", "content (hex)"] = 0
memory_map[["content (hex)"]]



* Now lets store the location of data to `0x6000 0000`.



In [ ]:
memory_map.loc["60000000", "content (hex)"] = "60000010"
memory_map[["content (hex)"]]



* Now we can say that `0x6000 0000` is pointing to `0x6000 0010`.
* Also a variable occupying `0x6000 0000` is a pointer variable.



|         | variable | pointer | reference |
|:------:|:--------:|:-------:|:---------:|
|declaration | `int i;` | `int *p;` | `int &r = i;` |
|  value  | `i` | `*p` | `r` |
|  address  | `&i` | `p` | `&r` |
|  address of address | N/A | `&p` | N/A |
  
  



## Example : a pointer to an integer variable



* Following C++ code shows declaring a pointer variable and assigining an address to it.[[ref](https://www.geeksforgeeks.org/pointers-vs-references-cpp/)]



In [ ]:
%%writefile int_ptr.cpp
#include <iostream>
#include <iomanip>

using namespace std;

int main(int argn, char *argv[]){
    // This is an integer variable
    int i = 0;

    // This is a pointer variable to the variable i
    int *p = &i;

    // Value of the variable i
    cout << "i         = " << setw(16) << hex <<  i << '\n';
    // Address of the variable i
    cout << "&i        = " << setw(16) << &i << '\n';
    // size of i
    cout << "sizeof(i) = " << setw(16) << sizeof(i) << '\n';

    // Integer value at the location p
    cout << "*p        = " << setw(16) << hex << *p << '\n';
    // Value of the pointer variable p
    cout << "p         = " << setw(16) <<  p << '\n';
    // Address of the pointer variable p
    cout << "&p        = " << setw(16) << &p << '\n';
    // Address of the pointer value (p + 1)
    cout << "(p + 1)   = " << setw(16) << (p + 1) << '\n';
    // Address of the pointer value (p + 2)
    cout << "(p + 2)   = " << setw(16) << (p + 2) << '\n';
    // size of p
    cout << "sizeof(p) = " << setw(16) << sizeof(p) << '\n';
    // Value at (p + 1)
    cout << "*(p + 1)  = " << setw(16) << *(p + 1) << '\n';
    // Value at (p + 2)
    cout << "*(p + 2)  = " << setw(16) << *(p + 2) << '\n';

    return 0;
}



In [ ]:
!g++ -Wall -g -std=c++14 int_ptr.cpp -o int_ptr -Wa,-adhln=int_ptr.s



In [ ]:
!./int_ptr



* Following is one of possible output from the program above.



```
i         =                0
&i        =   0x53c7ac35491c
sizeof(i) =                4
*p        =                0
p         =   0x53c7ac35491c
&p        =   0x53c7ac354920
(p + 1)   =   0x53c7ac354920
(p + 2)   =   0x53c7ac354924
sizeof(p) =                8
*(p + 1)  =         ac35491c
*(p + 2)  =             53c7
```



* Output above may imply a memory map as follows.



In [ ]:
base_addr = 0x53c7ac35491c

# Size of one variable : 4 Bytes == 32 bits
size_t = 4

# Number of 4 Byte variables
no_cell = 4


memory_map = pd.DataFrame(
    data={
        "index": range(no_cell)
    }
)

memory_map["offset"] = memory_map["index"] * size_t
memory_map["offset (hex)"] = memory_map["offset"].apply(lambda i:f"{i:02x}")
memory_map["address"] = memory_map["offset"].add(base_addr)
memory_map["address (hex)"] = memory_map["address"].apply(lambda i:f"{i:08x}")
memory_map["content"] = nr.randint(0xa000000, 0x7fffffff, size=(no_cell, 1))
memory_map["content (hex)"] = memory_map["content"].apply(lambda i:f"{i:08x}")

memory_map.set_index("address (hex)", inplace=True)

i_addr = f"{base_addr:08x}"
p_addr0 = f"{base_addr + size_t:08x}"
p_addr1 = f"{base_addr + size_t + size_t:08x}"

memory_map.loc[i_addr, "content (hex)"] = f"{0:08x}"
memory_map.loc[p_addr0, "content (hex)"] = f"{0xac35491c:08x}"
memory_map.loc[p_addr1, "content (hex)"] = f"{0x53c7:08x}"

memory_map[["content (hex)"]]



* How can you verify if this memory map is correct?



## Example : assigining to a pointer



* Following example shows assigning another address to a pointer variable after initialization.



In [ ]:
%%writefile ptr_asgn.cpp
#include <iostream>
#include <iomanip>

using namespace std;

int main(int argn, char *argv[]){
    // These are variables
    int i = 0, j = 0x01020304;

    // This is a pointer variable
    int *p = &i;

    // Value of the variable i
    cout << "i  = " << setw(16) << hex <<  i << '\n';
    // Address of the variable i
    cout << "&i = " << setw(16) << &i << '\n';

    // Integer value at the location p
    cout << "*p = " << setw(16) << hex << *p << '\n';
    // Value of the pointer variable p
    cout << "p  = " << setw(16) <<  p << '\n';
    // Address of the pointer variable p
    cout << "&p = " << setw(16) << &p << '\n';

    // Reassigning the pointer variable p
    p = &j;
    // Now p points to the variable j

    // Value of the variable j
    cout << "j  = " << setw(16) <<  j << '\n';
    // Address of the variable j
    cout << "&j = " << setw(16) << &j << '\n';

    // Integer value at the location p
    cout << "*p = " << setw(16) << *p << '\n';
    // Value of the pointer variable p
    cout << "p  = " << setw(16) <<  p << '\n';
    // Address of the pointer variable p
    cout << "&p = " << setw(16) << &p << '\n';
    
    return 0;
}



In [ ]:
!g++ -Wall -g -std=c++14 ptr_asgn.cpp -o ptr_asgn -Wa,-adhln=ptr_asgn.s



In [ ]:
!./ptr_asgn



## Example : a pointer and a reference



* Following example shows declaring a reference to a variable and assigning to it.



In [ ]:
%%writefile ptr_ref.cpp
#include <iostream>
#include <iomanip>

using namespace std;

int main(int argn, char *argv[]){
    // These are variables
    int i = 0;

    // This is a pointer variable
    int *p = &i;

    // This is a reference to i
    int &r = i;

    // Value of the variable i
    cout << "i  = " << setw(16) << hex <<  i << '\n';
    // Address of the variable i
    cout << "&i = " << setw(16) << &i << '\n';

    // Integer value at the location p
    cout << "*p = " << setw(16) << hex << *p << '\n';
    // Value of the pointer variable p
    cout << "p  = " << setw(16) <<  p << '\n';
    // Address of the pointer variable p
    cout << "&p = " << setw(16) << &p << '\n';

    // Value of reference variable r
    cout << "r  = " << setw(16) << hex << r << '\n';
    // Address of the reference variable r
    cout << "&r = " << setw(16) << &r << '\n';

    r = 0x01020304;

    // Value of reference variable r
    cout << "r  = " << setw(16) << hex << r << '\n';
    // Address of the reference variable r
    cout << "&r = " << setw(16) << &r << '\n';

    // Value of the variable i
    cout << "i  = " << setw(16) << hex <<  i << '\n';
    // Address of the variable i
    cout << "&i = " << setw(16) << &i << '\n';

    // Integer value at the location p
    cout << "*p = " << setw(16) << hex << *p << '\n';
    // Value of the pointer variable p
    cout << "p  = " << setw(16) <<  p << '\n';
    // Address of the pointer variable p
    cout << "&p = " << setw(16) << &p << '\n';
    
    return 0;
}



In [ ]:
!g++ -Wall -g -std=c++14 ptr_ref.cpp -o ptr_ref -Wa,-adhln=ptr_ref.s



In [ ]:
!./ptr_ref



## Pointer and array



* In C/C++, an array variable contains the start address of an array.



|         | variable | pointer | reference | array |
|:------:|:--------:|:-------:|:---------:|:---------:|
|declaration | `int i;` | `int *p;` | `int &r = i;` | `int a[] = {0, 1, 2, 3};` |
|  value  | `i` | `*p` | `r` | `a[i]` |
|  address  | `&i` | `p` | `&r` | `a+i` |
|  address of address | N/A | `&p` | N/A | `&a` |
  
  



* We can assign to an element of an array but not to the array



* Please consider following example.



In [ ]:
%%writefile ptr_ary.cpp
#include <iostream>
#include <iomanip>

using namespace std;

int main(int argn, char *argv[]){
    // This is an integer array
    int a[] = {1, 2, 3, 4};

    // This is a pointer variable to the array
    int *p = a;

    // Value of the first element of the array
    cout << "a[0] = " << setw(16) << hex <<  a[0] << '\n';
    // Address of the first element of the array
    cout << "a    = " << setw(16) << (a + 0) << '\n';
    // Address of the array variable;
    cout << "&a   = " << setw(16) << &a << '\n';

    // Integer value at the location p
    cout << "*p   = " << setw(16) << hex << *p << '\n';
    // Value of the pointer variable p
    cout << "p    = " << setw(16) <<  p << '\n';
    // Address of the pointer variable p
    cout << "&p   = " << setw(16) << &p << '\n';

    return 0;
}



In [ ]:
!g++ -Wall -g -std=c++14 ptr_ary.cpp -o ptr_ary -Wa,-adhln=ptr_ary.s



In [ ]:
!./ptr_ary



* Following example combines a for loop with the array and the pointers.



In [ ]:
%%writefile ptr_ary_for.cpp
#include <iostream>
#include <iomanip>

using namespace std;

int main(int argn, char *argv[]){
    // This is an integer array
    int a[] = {1, 2, 3, 4};

    // This is a pointer variable to the array
    int *p = a;
    int *p_inc = a;

    for (int i=0; 4 > i; ++i){


        // Value of the first element of the array
        cout << "a[" << i << "]     = " << setw(16) << hex <<  a[i] 
             << '\n';
        // Address of the first element of the array
        cout << "a + " << i << "    = " << setw(16) << (a + i) 
             << '\n';
        // Address of the array variable;
        cout << "&(a[" << i << "])  = " << setw(16) << &(a[i]) 
             << '\n';

        // Integer value at the location p + i
        cout << "*(p + " << i << ") = " << setw(16) << hex << *(p+i) 
             << '\n';
        // Address where p + i is pointing to
        cout << "p + " << i << "    = " << setw(16) <<  (p+i) 
             << '\n';

        // Integer value at the location p_inc
        cout << "*(p_inc) = " << setw(16) << hex << *(p_inc) 
             << '\n';
        // Value of the pointer variable p
        cout << "p_inc    = " << setw(16) <<  (p_inc) 
             << '\n';
        p_inc ++;
    }

    return 0;
}



In [ ]:
!g++ -Wall -g -std=c++14 ptr_ary_for.cpp -o ptr_ary_for -Wa,-adhln=ptr_ary_for.s



In [ ]:
!./ptr_ary_for



### Two-dimensional array



* Following example shows a two-dimensonal array.



* C/C++ compiler needs to know the size of the *column*.



In [ ]:
%%writefile ary_2d.cpp
#include <iostream>
#include <iomanip>

using namespace std;

int main(int argn, char *argv[]){
    
    // This is a two dimensional array
    int a[][4] = {
        {11, 12, 13, 14},
        {21, 22, 23, 24},
        {31, 32, 33, 34},
        {41, 42, 43, 44},
    };

    for (int i=0; 4 > i; ++i){
        for (int j=0; 4 > j; ++j){
            cout << a[i][j] << ' ';
        }        
        cout << '\n';
    }

    return 0;
}



In [ ]:
!g++ -Wall -g -std=c++14 ary_2d.cpp -o ary_2d -Wa,-adhln=ary_2d.s



In [ ]:
!./ary_2d



### Multi-dimensional array



* Following example shows a three-dimensonal array.



* C/C++ compiler needs to know all sizes except the first dimension.



In [ ]:
%%writefile ary_3d.cpp
#include <iostream>
#include <iomanip>

using namespace std;

int main(int argn, char *argv[]){

    // This is a three dimensional array
    int a[][2][3] = {
        {
            {111, 112, 113},
            {121, 122, 123},
        },
        {
            {211, 212, 213},
            {221, 222, 223},
        },
        {
            {311, 312, 313},
            {321, 322, 323},
        },
        {
            {411, 412, 413},
            {421, 422, 423},
        },
    };

    for (int i=0; 4 > i; ++i){
        for (int j=0; 2 > j; ++j){
            for (int k=0; 3 > k ; ++k){
                cout << a[i][j][k] << ' ';
            }
            cout << '\n';
        }
        cout << "============" << '\n';
    }

    return 0;
}



In [ ]:
!g++ -Wall -g -std=c++14 ary_3d.cpp -o ary_3d -Wa,-adhln=ary_3d.s



In [ ]:
!./ary_3d



## Array of Pointers



* An array may have pointers as its elements.



* Following example shows an array of `char` pointers.



In [ ]:
%%writefile ary_ptr.cpp
#include <iostream>

using namespace std;

int main(const int argn, char *argv[]){

    // This is an array of pointers
    const char *s[] = {
        "Is",
        "this",
        "sort of",
        "familiar ?",
    };

    for (int i=0; 4 > i; ++i){
            cout << s[i]<< ' ';
    }
    cout << '\n';

    // This is another array of pointers
    const char *t[] = {
        "If",
        "not",
        "certain",
        "run this with arguments",
    };

    for (int i=0; 4 > i; ++i){
            cout << t[i]<< ' ';
    }
    cout << '\n';

    // Please run with arguments
    for (int i=0; argn > i; ++i){
            cout << "argv[" << i << "]= " << argv[i]<< '\n';
    }

    return 0;
}



In [ ]:
!g++ -Wall -g -std=c++14 ary_ptr.cpp -o ary_ptr -Wa,-adhln=ary_ptr.s



In [ ]:
!./ary_ptr



## Calling by reference



* Basically, all C/C++ function calls are **call by value**; arguments are values not references.



* Changes of arguments within a *callee* function do not influence the *caller* function.



* However, there are cases that we need to update arguments.



* Following example shows how we can using both a pointer and a reference.



In [ ]:
%%writefile call_by_ref.cpp
#include <iostream>

using namespace std;

void accumulate_array_ptr(double *, const double);
void accumulate_array_ref(double &, const double);

int main(const int argn, char *argv[]){

    double update_this_with_pointer = 0.0;
    double update_this_with_reference = 0.0;

    for(int i=0; 10 > i; ++i){
        accumulate_array_ptr(&update_this_with_pointer, (double)i);
        accumulate_array_ref(update_this_with_reference, (double)i);
    }

    cout << "Pointer result   = " << update_this_with_pointer 
         << '\n';
    cout << "Reference result = " << update_this_with_reference 
         << '\n';

    return 0;
}

void accumulate_array_ptr(double * s, const double a){
    *s += a;
}

void accumulate_array_ref(double & t, const double a){
    t += a;
}



In [ ]:
!g++ -Wall -g -std=c++14 call_by_ref.cpp -o call_by_ref -Wa,-adhln=call_by_ref.s



In [ ]:
!./call_by_ref



## Exercise



### 00 Pointer



* Write a C++ code changing values of variables (`i`, `*p`, and `r` above) and observe the influences.



### 01 Multi-dimensional arrays



* Write a C++ code about $2\times2$ matrices as follows:



1. Write a function that accepts a `double [][2]` (or `double [2][2]`) and a `const char *` as arguments.<br>
This function would print the string and the $2\times2$ matrix.
1. Write a function that accepts three `double [][2]`'s (or `double [2][2]`'s) as arguments.<br>
The function would store sum of first two matricies to the third matrix.<br>If possible use `for` loops.
1. In the `main()` function, prepare three $2\times2$ matrices as two-dimensional arrays of `double`'s.<br>
Using the functions above, add two matrices of your choice and present the matrices before and after.<br>

